# **Base**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/AI Bioprocess/Data/~220915
!pwd
!ls
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/MyDrive/AI Bioprocess/Data/~220915
/content/drive/MyDrive/AI Bioprocess/Data/~220915
ammonia.csv	   input_glucose.csv	lactate.csv	    ph.csv
do.csv		   input_igg.csv	output_ammonia.csv  vcd.csv
glucose.csv	   input_lactate.csv	output_glucose.csv  viability.csv
igg.csv		   input_ph.csv		output_igg.csv	    y_pred_AIF_0.csv
input_ammonia.csv  input_vcd.csv	output_lactate.csv  y_test_0.csv
input_do.csv	   input_viability.csv	output_vcd.csv


In [ ]:
# 0:vcd, 2:glc, 3:lac, 4:amm, 7:igg
tn = 4

hp_svm     = [[5,2],[],[1,0.1],[1,0.1],[1,0.1],[],[],[1,0.1]]
hp_ann_lr  = [0.1, 0, 0.01,  0.03, 0.03, 0,0, 0.03]
hp_rnn_lr  = [0.1, 0, 0.008, 0.03, 0.03, 0,0, 0.03]
hp_lstm_lr = [0.5, 0, 0.5,   0.03, 0.03, 0,0, 0.03]
hp_gru_lr  = [2.5, 0, 0.1,   0.03, 0.03, 0,0, 0.03]

hp_lr = [0,0,0,0,0,0,0,0,0,0,0,0,hp_ann_lr[tn],hp_rnn_lr[tn],hp_lstm_lr[tn],hp_gru_lr[tn]]

units = ['($10^6$ cells/mL)',0,'(g/L)','(g/L)','(mM)',0,0,'(g/L)']

In [ ]:
import os
import random
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as rse

import matplotlib
import matplotlib.pyplot as plt

from itertools import combinations
import time
import warnings
from math import ceil

In [ ]:
# Functions

def seed_fix():
  seed = 1337
  os.environ['PYTHONHASHSEED'] = str(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = '1'
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  tf.compat.v1.set_random_seed(seed)
  config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
  K.set_session(sess)

# 모델 준비
seed_fix()
def build_LR():
  model = LinearRegression()
  return model

def build_LASSO():
  model = Lasso(alpha=0.03)
  return model

def build_EN():
  model = ElasticNet(alpha=0.03)
  return model

def build_KNN():
  model = KNeighborsRegressor()
  return model

def build_CART():
  model = DecisionTreeRegressor()
  return model

def build_GBM():
  model = MultiOutputRegressor(GradientBoostingRegressor())
  return model

def build_XG():
  model = MultiOutputRegressor(XGBRegressor())
  return model

def build_ADA():
  model = MultiOutputRegressor(AdaBoostRegressor())
  return model

def build_SVM():
  model = MultiOutputRegressor(SVR(C=hp_svm[tn][0],epsilon = hp_svm[tn][1]))
  return model

def build_RF():
  model = RandomForestRegressor()
  return model

def build_PLS():
  model = PLSRegression()
  return model

def build_GPR():
  kernel = DotProduct() + WhiteKernel()
  model = GaussianProcessRegressor(kernel=kernel)
  return model

def build_ANN(input_shape,layer_1,layer_2,output_num,Learning_rate):
  seed=1337
  model = keras.Sequential([
                            layers.Dense(layer_1, activation='sigmoid', kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(input_shape,)),
                            layers.Dense(layer_2, activation='sigmoid', kernel_initializer=keras.initializers.HeNormal(seed=seed)),
                            layers.Dense(output_num)
                            ])
  
  optimizer = tf.keras.optimizers.RMSprop(Learning_rate)
  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mae','mse']
                )
  return model

def build_RNN(input_feature,units,output_feature,time_step,Learning_rate):
  seed=1337
  model = keras.Sequential()
  model.add(keras.layers.SimpleRNN(units,kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(time_step,input_feature),stateful=False))
  model.add(keras.layers.Dropout(rate=0.1))
  model.add(keras.layers.Dense(units=output_feature,kernel_initializer=keras.initializers.HeNormal(seed=seed)))
  optimizer = tf.keras.optimizers.Adam(learning_rate=Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
  model.compile(loss='mae', optimizer=optimizer)
  return model

def build_LSTM(input_feature,units,output_feature,time_step,Learning_rate):
  seed=1337
  model = keras.Sequential()
  model.add(keras.layers.LSTM(units,kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(time_step,input_feature),stateful=False))
  model.add(keras.layers.Dropout(rate=0.1))
  model.add(keras.layers.Dense(units=output_feature,kernel_initializer=keras.initializers.HeNormal(seed=seed)))
  optimizer = tf.keras.optimizers.Adam(learning_rate=Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
  model.compile(loss='mae', optimizer=optimizer)
  return model

def build_GRU(input_feature,units,output_feature,time_step,Learning_rate):
  seed=1337
  model = keras.Sequential()
  model.add(keras.layers.GRU(units,kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(time_step,input_feature),stateful=False))
  model.add(keras.layers.Dropout(rate=0.1))
  model.add(keras.layers.Dense(units=output_feature,kernel_initializer=keras.initializers.HeNormal(seed=seed)))
  optimizer = tf.keras.optimizers.Adam(learning_rate=Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
  model.compile(loss='mae', optimizer=optimizer)
  return model

def make_recursive(data):
  X = np.zeros((data.shape[0]*(data.shape[1]-3),3))
  y = np.zeros((data.shape[0]*(data.shape[1]-3),1))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-3):
      X[(5*i+j):(5*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
      y[(5*i+j):(5*i+j+1),0:1] = data[i:(i+1),(j+3):(j+4)]
  return X.astype(float), y.astype(float)

def make_direct(data,future):
  X = np.zeros((data.shape[0]*(data.shape[1]-2-future),3))
  y = np.zeros((data.shape[0]*(data.shape[1]-2-future),1))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-2-future):
      X[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
      y[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:1] = data[i:(i+1),(j+2+future):(j+3+future)]
  return X.astype(float), y.astype(float)

def make_mimo(data):
  X = np.zeros((data.shape[0],4))
  y = np.zeros((data.shape[0],4))
  for i in range(data.shape[0]):
    X[i:(i+1),0:4] = data[i:(i+1),0:4]
    y[i:(i+1),0:4] = data[i:(i+1),4:8]
  return X.astype(float), y.astype(float)

def make_dirrec(data,future):
  X = np.zeros((data.shape[0]*(data.shape[1]-2-future),2+future))
  y = np.zeros((data.shape[0]*(data.shape[1]-2-future),1))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-2-future):
      X[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:(2+future)] = data[i:(i+1),j:(j+future+2)]
      y[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:1] = data[i:(i+1),(j+2+future):(j+3+future)]
  return X.astype(float), y.astype(float)

def make_dirmo3(data):
  X = np.zeros((data.shape[0]*(data.shape[1]-5),3))
  y = np.zeros((data.shape[0]*(data.shape[1]-5),3))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-5):
      X[((data.shape[1]-5)*i+j):((data.shape[1]-5)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
      y[((data.shape[1]-5)*i+j):((data.shape[1]-5)*i+j+1),0:3] = data[i:(i+1),(j+3):(j+6)]
  return X.astype(float), y.astype(float)

def make_dirmo2(data):
  X = np.zeros((data.shape[0]*(data.shape[1]-7),3))
  y = np.zeros((data.shape[0]*(data.shape[1]-7),2))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-7):
      X[((data.shape[1]-7)*i+j):((data.shape[1]-7)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
      y[((data.shape[1]-7)*i+j):((data.shape[1]-7)*i+j+1),0:2] = data[i:(i+1),(j+6):(j+8)]
  return X.astype(float), y.astype(float)

def algorithm_selection(a,input_num,output_num,input_feature,units,output_feature,time_step,Learning_rate):
  if a == 0:
    model = build_LR()
  elif a == 1:
    model = build_LASSO()
  elif a == 2:
    model = build_EN()
  elif a == 3:
    model = build_KNN()
  elif a == 4:
    model = build_CART()
  elif a == 5:
    model = build_GBM()
  elif a == 6:
    model = build_XG()
  elif a == 7:
    model = build_ADA()
  elif a == 8:
    model = build_SVM()
  elif a == 9:
    model = build_RF()
  elif a == 10:
    model = build_PLS()
  elif a == 11:
    model = build_GPR()
  elif a == 12:
    model = build_ANN(input_num,input_num,ceil(input_num/2),output_num,Learning_rate)
  elif a == 13:
    model = build_RNN(input_feature,units,output_feature,time_step,Learning_rate)
  elif a == 14:
    model = build_LSTM(input_feature,units,output_feature,time_step,Learning_rate)
  elif a == 15:
    model = build_GRU(input_feature,units,output_feature,time_step,Learning_rate)
  return model


def recursive_predict(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  #recursive
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_recursive(data_train_input[i])[0])
    y_train_stack.append(make_recursive(data_train_output[i])[1])
    X_test_stack.append(make_recursive(data_test_input[i])[0])
    y_test_stack.append(make_recursive(data_test_output[i])[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_recursive(data_train_output[len(all_vcd_com)])[1])
    y_test_stack.append(make_recursive(data_test_output[len(all_vcd_com)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  if mn != 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn == 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)
  
  address1 = []
  address2 = []
  address1 = list(np.arange(0,out_list.index(tn)))
  address2 = list(np.arange(out_list.index(tn)+len(out_list)-len(all_vcd_com), len(out_list)))
  address1.extend(address2)

  conditions = []
  conditions.append(X_test_list[0].reshape(20,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[5,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[10,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[15,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    VCD_tem[cn] = np.concatenate([VCD_tem[cn],np.zeros(shape=(VCD_tem[cn].shape[0],8-VCD_tem[cn].shape[1]))],1)
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)


  #recursive
  for cn in range(4):
    for day in range(5):
      VCD_tem[cn][:,(day+3):(day+4)] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,VCD_tem[cn].shape[0]*3))).T[address1]
      y_pred_all[cn][day+3] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,VCD_tem[cn].shape[0]*3))).T[out_list.index(tn)]

  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


def direct_predict(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []
  # direct 1
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],1)[0])
    X_test_stack.append(make_direct(data_test_input[i],1)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],1)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],1)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],2)[0])
    X_test_stack.append(make_direct(data_test_input[i],2)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],2)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],2)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 3
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],3)[0])
    X_test_stack.append(make_direct(data_test_input[i],3)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],3)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],3)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 4
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],4)[0])
    X_test_stack.append(make_direct(data_test_input[i],4)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],4)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],4)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 5
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],5)[0])
    X_test_stack.append(make_direct(data_test_input[i],5)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],5)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],5)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  if mn != 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn == 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  conditions.append(X_test_list[0].reshape(20,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[5,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[10,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[15,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  # direct
  for cn in range(4):
    for pm in range(5):
      y_pred_all[cn][pm+3] = models[pm].predict(scaler_list[pm].transform(VCD_tem[cn][:,0:3].reshape(1,VCD_tem[cn].shape[0]*3))).reshape(1)

  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


def mimo_predict(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # MIMO
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_mimo(data_train_input[i])[0])
    X_test_stack.append(make_mimo(data_test_input[i])[0])
  y_train_stack.append(make_mimo(data_train_output[out_list.index(tn)])[1])
  y_test_stack.append(make_mimo(data_test_output[out_list.index(tn)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  if mn != 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn == 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  conditions.append(X_test_list[0].reshape(4,-1,4)[0,:,:])
  conditions.append(X_test_list[0].reshape(4,-1,4)[1,:,:])
  conditions.append(X_test_list[0].reshape(4,-1,4)[2,:,:])
  conditions.append(X_test_list[0].reshape(4,-1,4)[3,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:5])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:5])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:5])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:5])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  # MIMO
  for cn in range(4):
    y_pred_all[cn][4:8] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,0:4].reshape(1,VCD_tem[cn].shape[0]*4))).reshape(4)

  return y_pred_all, np.concatenate([y_pred_all[0][4:8],y_pred_all[1][4:8],y_pred_all[2][4:8],y_pred_all[3][4:8]])



def dirrec_predict(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []
  # dirrec 1
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],1)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],1)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],1)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],1)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],1)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],1)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],2)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],2)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],2)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],2)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],2)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],2)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 3
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],3)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],3)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],3)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],3)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],3)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],3)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 4
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],4)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],4)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],4)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],4)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],4)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],4)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 5
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],5)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],5)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],5)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],5)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],5)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],5)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  if mn != 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn == 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)
  
  address1 = []
  address2 = []
  address1 = list(np.arange(0,out_list.index(tn)))
  address2 = list(np.arange(out_list.index(tn)+len(out_list)-len(all_vcd_com), len(out_list)))
  address1.extend(address2)

  conditions = []
  conditions.append(X_test_list[0].reshape(20,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[5,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[10,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[15,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    VCD_tem[cn] = np.concatenate([VCD_tem[cn],np.zeros(shape=(VCD_tem[cn].shape[0],8-VCD_tem[cn].shape[1]))],1)
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)
  for cn in range(4):
    for day in range(5):
      VCD_tem[cn][:,(day+3):(day+4)] = models[day].predict(scaler_list[day].transform(VCD_tem[cn][:,0:(day+3)].reshape(1,VCD_tem[cn].shape[0]*(3+day)))).T[address1]
      y_pred_all[cn][day+3] = models[day].predict(scaler_list[day].transform(VCD_tem[cn][:,0:(day+3)].reshape(1,VCD_tem[cn].shape[0]*(3+day)))).T[out_list.index(tn)]

  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


def dirmo_predict(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # dirmo 1
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirmo3(data_train_input[i])[0])
    X_test_stack.append(make_dirmo3(data_test_input[i])[0])
  y_train_stack.append(make_dirmo3(data_train_output[out_list.index(tn)])[1])
  y_test_stack.append(make_dirmo3(data_test_output[out_list.index(tn)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirmo2(data_train_input[i])[0])
    X_test_stack.append(make_dirmo2(data_test_input[i])[0])
  y_train_stack.append(make_dirmo2(data_train_output[out_list.index(tn)])[1])
  y_test_stack.append(make_dirmo2(data_test_output[out_list.index(tn)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  if mn != 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn == 12:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  conditions.append(X_test_list[0].reshape(12,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(12,-1,3)[3,:,:])
  conditions.append(X_test_list[0].reshape(12,-1,3)[6,:,:])
  conditions.append(X_test_list[0].reshape(12,-1,3)[9,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  # dirmo
  for cn in range(4):
    y_pred_all[cn][3:6] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,0:3].reshape(1,VCD_tem[cn].shape[0]*3))).reshape(3)[0]
    y_pred_all[cn][6:8] = models[1].predict(scaler_list[1].transform(VCD_tem[cn][:,0:3].reshape(1,VCD_tem[cn].shape[0]*3))).reshape(2)[0]
  
  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


##########################################################################################################
##########################################################################################################
##########################################################################################################

def recursive_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  #recursive
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_recursive(data_train_input[i])[0])
    y_train_stack.append(make_recursive(data_train_output[i])[1])
    X_test_stack.append(make_recursive(data_test_input[i])[0])
    y_test_stack.append(make_recursive(data_test_output[i])[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_recursive(data_train_output[len(all_vcd_com)])[1])
    y_test_stack.append(make_recursive(data_test_output[len(all_vcd_com)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  for i in range(len(X_train_list)):
    X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
    X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  for i in range(len(X_train_s_list)):
    seed_fix()
    models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)
  
  address1 = []
  address2 = []
  address1 = list(np.arange(0,out_list.index(tn)))
  address2 = list(np.arange(out_list.index(tn)+len(out_list)-len(all_vcd_com), len(out_list)))
  address1.extend(address2)

  conditions = []
  conditions.append(X_test_list[0].reshape(20,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[5,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[10,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[15,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    VCD_tem[cn] = np.concatenate([VCD_tem[cn],np.zeros(shape=(VCD_tem[cn].shape[0],8-VCD_tem[cn].shape[1]))],1)
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)


  #recursive
  for cn in range(4):
    for day in range(5):
      VCD_tem[cn][:,(day+3):(day+4)] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,-1)).reshape(1,VCD_tem[cn][:,day:(day+3)].shape[0],VCD_tem[cn][:,day:(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[address1]
      y_pred_all[cn][day+3] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,-1)).reshape(1,VCD_tem[cn][:,day:(day+3)].shape[0],VCD_tem[cn][:,day:(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[out_list.index(tn)][0]

  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


def direct_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []
  # direct 1
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],1)[0])
    X_test_stack.append(make_direct(data_test_input[i],1)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],1)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],1)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],2)[0])
    X_test_stack.append(make_direct(data_test_input[i],2)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],2)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],2)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 3
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],3)[0])
    X_test_stack.append(make_direct(data_test_input[i],3)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],3)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],3)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 4
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],4)[0])
    X_test_stack.append(make_direct(data_test_input[i],4)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],4)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],4)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 5
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],5)[0])
    X_test_stack.append(make_direct(data_test_input[i],5)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],5)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],5)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  for i in range(len(X_train_list)):
    X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
    X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  for i in range(len(X_train_s_list)):
    seed_fix()
    models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  conditions.append(X_test_list[0].reshape(20,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[5,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[10,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[15,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  # direct
  for cn in range(4):
    for pm in range(5):
      y_pred_all[cn][pm+3] = models[pm].predict(scaler_list[pm].transform(VCD_tem[cn][:,0:3].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:3].shape[0],VCD_tem[cn][:,0:3].shape[1]).transpose(0,2,1),verbose=0).reshape(1)[0]

  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


def mimo_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # MIMO
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_mimo(data_train_input[i])[0])
    X_test_stack.append(make_mimo(data_test_input[i])[0])
  y_train_stack.append(make_mimo(data_train_output[out_list.index(tn)])[1])
  y_test_stack.append(make_mimo(data_test_output[out_list.index(tn)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  for i in range(len(X_train_list)):
    X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
    X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  for i in range(len(X_train_s_list)):
    seed_fix()
    models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  conditions.append(X_test_list[0].reshape(4,-1,4)[0,:,:])
  conditions.append(X_test_list[0].reshape(4,-1,4)[1,:,:])
  conditions.append(X_test_list[0].reshape(4,-1,4)[2,:,:])
  conditions.append(X_test_list[0].reshape(4,-1,4)[3,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:5])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:5])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:5])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:5])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  # MIMO
  for cn in range(4):
    y_pred_all[cn][4:8] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,0:4].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:4].shape[0],VCD_tem[cn][:,0:4].shape[1]).transpose(0,2,1),verbose=0).reshape(4)

  return y_pred_all, np.concatenate([y_pred_all[0][4:8],y_pred_all[1][4:8],y_pred_all[2][4:8],y_pred_all[3][4:8]])



def dirrec_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []
  # dirrec 1
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],1)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],1)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],1)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],1)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],1)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],1)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],2)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],2)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],2)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],2)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],2)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],2)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 3
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],3)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],3)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],3)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],3)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],3)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],3)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 4
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],4)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],4)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],4)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],4)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],4)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],4)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 5
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],5)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],5)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],5)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],5)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],5)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],5)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  for i in range(len(X_train_list)):
    X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
    X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  for i in range(len(X_train_s_list)):
    seed_fix()
    models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)
  
  address1 = []
  address2 = []
  address1 = list(np.arange(0,out_list.index(tn)))
  address2 = list(np.arange(out_list.index(tn)+len(out_list)-len(all_vcd_com), len(out_list)))
  address1.extend(address2)

  conditions = []
  conditions.append(X_test_list[0].reshape(20,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[5,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[10,:,:])
  conditions.append(X_test_list[0].reshape(20,-1,3)[15,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    VCD_tem[cn] = np.concatenate([VCD_tem[cn],np.zeros(shape=(VCD_tem[cn].shape[0],8-VCD_tem[cn].shape[1]))],1)
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)
  for cn in range(4):
    for day in range(5):
      VCD_tem[cn][:,(day+3):(day+4)] = models[day].predict(scaler_list[day].transform(VCD_tem[cn][:,0:(day+3)].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:(day+3)].shape[0],VCD_tem[cn][:,0:(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[address1]
      y_pred_all[cn][day+3] = models[day].predict(scaler_list[day].transform(VCD_tem[cn][:,0:(day+3)].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:(day+3)].shape[0],VCD_tem[cn][:,0:(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[out_list.index(tn)][0]
  
  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


def dirmo_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # dirmo 1
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirmo3(data_train_input[i])[0])
    X_test_stack.append(make_dirmo3(data_test_input[i])[0])
  y_train_stack.append(make_dirmo3(data_train_output[out_list.index(tn)])[1])
  y_test_stack.append(make_dirmo3(data_test_output[out_list.index(tn)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirmo2(data_train_input[i])[0])
    X_test_stack.append(make_dirmo2(data_test_input[i])[0])
  y_train_stack.append(make_dirmo2(data_train_output[out_list.index(tn)])[1])
  y_test_stack.append(make_dirmo2(data_test_output[out_list.index(tn)])[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  for i in range(len(X_train_list)):
    X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
    X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],hp_lr[mn]))

  for i in range(len(X_train_s_list)):
    seed_fix()
    models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  conditions.append(X_test_list[0].reshape(12,-1,3)[0,:,:])
  conditions.append(X_test_list[0].reshape(12,-1,3)[3,:,:])
  conditions.append(X_test_list[0].reshape(12,-1,3)[6,:,:])
  conditions.append(X_test_list[0].reshape(12,-1,3)[9,:,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:4])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:4])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  # dirmo
  for cn in range(4):
    y_pred_all[cn][3:6] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,0:3].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:3].shape[0],VCD_tem[cn][:,0:3].shape[1]).transpose(0,2,1),verbose=0).reshape(3)
    y_pred_all[cn][6:8] = models[1].predict(scaler_list[1].transform(VCD_tem[cn][:,0:3].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:3].shape[0],VCD_tem[cn][:,0:3].shape[1]).transpose(0,2,1),verbose=0).reshape(2)

  
  return y_pred_all, np.concatenate([y_pred_all[0][3:8],y_pred_all[1][3:8],y_pred_all[2][3:8],y_pred_all[3][3:8]])


##########################################################################################################
##########################################################################################################
##########################################################################################################


def forecasting_selection(RNN,f,all_vcd_com,mn,raw_data,condition_train,condition_test):
  if RNN == 0 and f == 0:
    return recursive_predict(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 0 and f == 1:
    return direct_predict(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 0 and f == 2:
    return mimo_predict(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 0 and f == 3:
    return dirrec_predict(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 0 and f == 4:
    return dirmo_predict(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 1 and f == 0:
    return recursive_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 1 and f == 1:
    return direct_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 1 and f == 2:
    return mimo_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 1 and f == 3:
    return dirrec_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test)
  elif RNN == 1 and f == 4:
    return dirmo_predict_RNN(all_vcd_com,mn,raw_data,condition_train,condition_test)


# **CODE**

In [ ]:
start = time.time()  # 시작 시간 저장
warnings.filterwarnings(action='ignore')

### data 준비 ###
# input
seed_fix()
data_input = []
data_input.append(pd.read_csv("input_vcd.csv"))
data_input.append(pd.read_csv("input_viability.csv"))
data_input.append(pd.read_csv("input_glucose.csv"))
data_input.append(pd.read_csv("input_lactate.csv"))
data_input.append(pd.read_csv("input_ammonia.csv"))
data_input.append(pd.read_csv("input_do.csv"))
data_input.append(pd.read_csv("input_ph.csv"))
data_input.append(pd.read_csv("input_igg.csv"))

# output
data_output = []
data_output.append(pd.read_csv("output_vcd.csv"))
data_output.append(pd.read_csv("input_viability.csv"))
data_output.append(pd.read_csv("output_glucose.csv"))
data_output.append(pd.read_csv("output_lactate.csv"))
data_output.append(pd.read_csv("output_ammonia.csv"))
data_output.append(pd.read_csv("input_do.csv"))
data_output.append(pd.read_csv("input_ph.csv"))
data_output.append(pd.read_csv("output_igg.csv"))

# multi-step
raw_data = []
raw_data.append(pd.read_csv('vcd.csv'))
raw_data.append(pd.read_csv('viability.csv'))
raw_data.append(pd.read_csv('glucose.csv'))
raw_data.append(pd.read_csv('lactate.csv'))
raw_data.append(pd.read_csv('ammonia.csv'))
raw_data.append(pd.read_csv('do.csv'))
raw_data.append(pd.read_csv('ph.csv'))
raw_data.append(pd.read_csv('igg.csv'))

# test batches for each condition
condition_test1 = "P1_1"
condition_test2 = "P2_2"
condition_test3 = "P3_5"
condition_test4 = "P4_6"
condition_test_os  = (data_input[0].condition == condition_test1) + (data_input[0].condition == condition_test2) + (data_input[0].condition == condition_test3) + (data_input[0].condition == condition_test4)
condition_train_os = ~condition_test_os
condition_train_ms = ~((raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4))
condition_test_ms  = (raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4)

mimo_list = list(range(20))
for i in range(4):
  mimo_list.remove(5*i)

# permutation item and etc..
item = [0,1,2,3,4,5,6,7]
model_name=np.array(["Linear","Lasso","EN","KNN","CART","GBM","XGB","Ada","SVM","RF","PLS","GPR","ANN","RNN","LSTM","GRU"])
forecasting_strategy = np.array(["One-step","Recursive",'Direct','MIMO','DirRec','DIRMO'])

error_table_mae  = []
error_table_mape = []
error_table_mse  = []
error_table_rmse = []
error_table_rse  = []
y_pred_table = []

com_list = []
for k in range(8):
  com = list(combinations(item,k+1))
  for i in range(len(com)):
    com_list.append(com[i])

  
# One-step ahead forecasting
y_train = data_output[tn][condition_train_os].to_numpy()[:,1:2]
y_test  = data_output[tn][condition_test_os].to_numpy()[:,1:2]


mae_list  = []
mape_list = []
mse_list  = []
rmse_list = []
rse_list  = []
pred_list = []
for mn in range(16):
  mae_list.append([])
  mape_list.append([])
  mse_list.append([])
  rmse_list.append([])
  rse_list.append([])
  pred_list.append([])

models_list = []
for i in range(8):
  models_list.append([])
  for mn in range(16):
    models_list[i].append(algorithm_selection(mn,3*(i+1),1,i+1,i+1,y_train.shape[1],3,hp_lr[mn]))

# for i in range(len(com_list)):
for i in range(len(com_list)):
  X_train_list = []
  X_test_list = []
  for j in com_list[i]:
    X_train_list.append(data_input[j][condition_train_os].to_numpy()[:,1:4])
    X_test_list.append(data_input[j][condition_test_os].to_numpy()[:,1:4])
  X_train = np.concatenate(X_train_list,1)
  X_test = np.concatenate(X_test_list,1)
  scaler = StandardScaler()
  X_train_s = scaler.fit_transform(X_train)
  X_test_s = scaler.transform(X_test)
  X_train_s_rs = X_train_s.reshape(X_train_s.shape[0],-1,3).transpose(0,2,1)
  X_test_s_rs  = X_test_s.reshape(X_test_s.shape[0],-1,3).transpose(0,2,1)

  models = []
  models = models_list[len(com_list[i])-1].copy()

  for mn in range(12):
    seed_fix()
    models[mn].fit(X_train_s.astype(float),y_train.astype(float))
  seed_fix()
  models[12].fit(X_train_s.astype(float),y_train.astype(float),epochs=100,verbose=0)
  for mn in [13,14,15]:
    seed_fix()
    models[mn].fit(X_train_s_rs.astype(float),y_train.astype(float),epochs=100,verbose=0)

  for mn in range(13):
    mae_list[mn].append(mae(y_test,models[mn].predict(X_test_s)))
    mape_list[mn].append(mape(y_test,models[mn].predict(X_test_s)))
    mse_list[mn].append(mse(y_test,models[mn].predict(X_test_s)))
    rmse_list[mn].append(mse(y_test,models[mn].predict(X_test_s),squared=False))
    rse_list[mn].append(rse(y_test,models[mn].predict(X_test_s)))
    pred_list[mn].append(models[mn].predict(X_test_s))
  for mn in [13,14,15]:
    mae_list[mn].append(mae(y_test,models[mn].predict(X_test_s_rs,verbose=0)))
    mape_list[mn].append(mape(y_test,models[mn].predict(X_test_s_rs,verbose=0)))
    mse_list[mn].append(mse(y_test,models[mn].predict(X_test_s_rs,verbose=0)))
    rmse_list[mn].append(mse(y_test,models[mn].predict(X_test_s_rs,verbose=0),squared=False))
    rse_list[mn].append(rse(y_test,models[mn].predict(X_test_s_rs,verbose=0)))
    pred_list[mn].append(models[mn].predict(X_test_s_rs,verbose=0))

error_table_mae.append(mae_list)
error_table_mape.append(mape_list)
error_table_mse.append(mse_list)
error_table_rmse.append(rmse_list)
error_table_rse.append(rse_list)
y_pred_table.append(pred_list)


# Multi-step ahead forecasting
y_test = np.concatenate([raw_data[tn][condition_test_ms].to_numpy()[0,4:9],raw_data[tn][condition_test_ms].to_numpy()[1,4:9],raw_data[tn][condition_test_ms].to_numpy()[2,4:9],raw_data[tn][condition_test_ms].to_numpy()[3,4:9]])
y_test_mimo = y_test[mimo_list]

for forecasting_n in range(5):
  mae_list  = []
  mape_list = []
  mse_list  = []
  rmse_list = []
  rse_list  = []
  pred_list = []
  for mn in range(16):
    mae_list.append([])
    mape_list.append([])
    mse_list.append([])
    rmse_list.append([])
    rse_list.append([])
    pred_list.append([])
  # for i in range(len(com_list)):
  for i in range(len(com_list)):
    for mn in range(16):
      y_pred = forecasting_selection(mn//13,forecasting_n,com_list[i],mn,raw_data,condition_train_ms,condition_test_ms)[1]
      pred_list[mn].append(y_pred)
      if forecasting_n==2:
        mae_list[mn].append(mae(y_test_mimo,y_pred))
        mape_list[mn].append(mape(y_test_mimo,y_pred))
        mse_list[mn].append(mse(y_test_mimo,y_pred))
        rmse_list[mn].append(mse(y_test_mimo,y_pred,squared=False))
        rse_list[mn].append(rse(y_test_mimo,y_pred))        
      else:
        mae_list[mn].append(mae(y_test,y_pred))
        mape_list[mn].append(mape(y_test,y_pred))
        mse_list[mn].append(mse(y_test,y_pred))
        rmse_list[mn].append(mse(y_test,y_pred,squared=False))
        rse_list[mn].append(rse(y_test,y_pred))

  error_table_mae.append(mae_list)
  error_table_mape.append(mape_list)
  error_table_mse.append(mse_list)
  error_table_rmse.append(rmse_list)
  error_table_rse.append(rse_list)
  y_pred_table.append(pred_list)

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 28ms/step
[08:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1/1 [==============================] - 0s 27ms/step
[08:24:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[08:24:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated i

In [ ]:
%cd /content/drive/MyDrive/AI Bioprocess/colab note/~221019/result/
!pwd

df = pd.DataFrame(np.concatenate((np.array(error_table_mae[0]).T,
                                  np.array(error_table_mae[1]).T,
                                  np.array(error_table_mae[2]).T,
                                  np.array(error_table_mae[3]).T,
                                  np.array(error_table_mae[4]).T,
                                  np.array(error_table_mae[5]).T),1))
df.to_csv('MAE_'+str(tn)+'.csv',index=False)

df = pd.DataFrame(np.concatenate((np.array(error_table_mape[0]).T,
                                  np.array(error_table_mape[1]).T,
                                  np.array(error_table_mape[2]).T,
                                  np.array(error_table_mape[3]).T,
                                  np.array(error_table_mape[4]).T,
                                  np.array(error_table_mape[5]).T),1))
df.to_csv('MAPE_'+str(tn)+'.csv',index=False)

df = pd.DataFrame(np.concatenate((np.array(error_table_mse[0]).T,
                                  np.array(error_table_mse[1]).T,
                                  np.array(error_table_mse[2]).T,
                                  np.array(error_table_mse[3]).T,
                                  np.array(error_table_mse[4]).T,
                                  np.array(error_table_mse[5]).T),1))
df.to_csv('MSE_'+str(tn)+'.csv',index=False)

df = pd.DataFrame(np.concatenate((np.array(error_table_rmse[0]).T,
                                  np.array(error_table_rmse[1]).T,
                                  np.array(error_table_rmse[2]).T,
                                  np.array(error_table_rmse[3]).T,
                                  np.array(error_table_rmse[4]).T,
                                  np.array(error_table_rmse[5]).T),1))
df.to_csv('RMSE_'+str(tn)+'.csv',index=False)

df = pd.DataFrame(np.concatenate((np.array(error_table_rse[0]).T,
                                  np.array(error_table_rse[1]).T,
                                  np.array(error_table_rse[2]).T,
                                  np.array(error_table_rse[3]).T,
                                  np.array(error_table_rse[4]).T,
                                  np.array(error_table_rse[5]).T),1))
df.to_csv('RSE_'+str(tn)+'.csv',index=False)

for f in range(6):
  df = pd.DataFrame(np.concatenate((np.array(y_pred_table[f][0]).reshape(255,-1),
                                    np.array(y_pred_table[f][1]).reshape(255,-1),
                                    np.array(y_pred_table[f][2]).reshape(255,-1),
                                    np.array(y_pred_table[f][3]).reshape(255,-1),
                                    np.array(y_pred_table[f][4]).reshape(255,-1),
                                    np.array(y_pred_table[f][5]).reshape(255,-1),
                                    np.array(y_pred_table[f][6]).reshape(255,-1),
                                    np.array(y_pred_table[f][7]).reshape(255,-1),
                                    np.array(y_pred_table[f][8]).reshape(255,-1),
                                    np.array(y_pred_table[f][9]).reshape(255,-1),
                                    np.array(y_pred_table[f][10]).reshape(255,-1),
                                    np.array(y_pred_table[f][11]).reshape(255,-1)),1))
  
  df.to_csv(forecasting_strategy[f]+'_'+str(tn)+'.csv',index=False)

pd.DataFrame(y_test).to_csv('y_test_'+str(tn)+'.csv',index=False)